**Impact of minority shareholdings on innovation**

The first steps of our research agenda were to understand, choose and clean the data we would later use for regression. 
This notebook mainly deals with the creation of the database. We used data from Orbis and from Zephir.

**Collection and cleaning of the data from Zephir** \
From Zephir, we extracted all minority deals (with pieces of information about it) data regarding deals made by companies. \
Minority deals are deals with an acquired stake below 50%. 
As we could only download 10.000 rows by 10.000 rows, we had to concatenate 16 small databases. They are is the 'data_eco' file. 
We took advantage of this first step for converting the date. 

In [1]:
# importation 

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

In [2]:
# concaténation de tout les fichiers + nettoyage sommaire des données
path = "data_eco"
folder = os.listdir(path)
files_xls = []
def date_parser(x):
    if type(x) == int:
        y = pd.to_datetime(pd.to_datetime(x, unit='D', origin='1899-12-30').strftime(format='%d-%m-%Y'))
    else:
        y = pd.to_datetime(x, dayfirst=True)
    return y
for file in folder :
    files_xls.append(f'{path}/{file}')
print(files_xls)
df = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel(f, 'Results', converters={'Completed date':date_parser, 'Assumed completion date':date_parser })
    df = df.append(data)

['data_eco/MS_final16.xlsx', 'data_eco/MS_final2.xlsx', 'data_eco/MS_final3.xlsx', 'data_eco/MS_final8.xlsx', 'data_eco/MS_final10.xlsx', 'data_eco/MS_final4.xlsx', 'data_eco/MS_final5.xlsx', 'data_eco/MS_final11.xlsx', 'data_eco/MS_final9.xlsx', 'data_eco/MS_final6.xlsx', 'data_eco/MS_final12.xlsx', 'data_eco/MS_final13.xlsx', 'data_eco/MS_final7.xlsx', 'data_eco/MS_final14.xlsx', 'data_eco/MS_final15.xlsx', 'data_eco/MS_final1.xlsx']


Columns of this database are : 

In [3]:
# save of the concateta databases. 
#df.to_csv('data.csv', index = True)
df = pd.read_csv('data.csv')

/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


The **second step** was to clean the data. \
1_ We had to remove the rows that were empty. \
2_ Verifie wether the deal acquired stake were actualy below 50%, and remove the row if not. \
3_ Remove acquisitions with a missing date. 2 different dates were mentionned in the database. When "completion date" was mentionned, "Assumed completion date" wasn't and vice versa. We decided to aggragate those 2 columns in "Completed or assumed date" \
4_ Remove acquisitions where the seller ("Acquiror") and the "Target" were one and the same company. \
5_ Remove acquisitions with missing data on the target or acquiror's country code. 
6_ Finnaly, we only kept domestic acquisitions by removing acquisition with different target country code and acquiror contry code. 

In [4]:
shape_initiale = df.shape[0]
#supprimer les lignes vident de données
df.dropna(axis=0, subset=['Unnamed: 0','Target BvD ID number', 'Acquiror BvD ID number',"Acquired stake (%)" ], how='any', inplace=True)
shape_with_empty_rows = shape_initiale - df.shape[0]
shape_without_empty_rows = df.shape[0]

#Vérification de si les acquired stake sont bien inférieurs à 50%

df = df[df["Acquired stake (%)"] != "Unknown %"]
shape_with_unknown_acquired_stake = shape_without_empty_rows - df.shape[0]
shape_without_unknown_acquired_stake = df.shape[0]

def replace(x):
    if type(x)==str :
        if "Unknown majority" in x:
            x = str(60)
        elif 'Unknown minority' in x:
            x = str(1)
        if '\n' in x:
            x = str(60)
    
    x = float(x)
    return x
df["Acquired stake (%)"] = df["Acquired stake (%)"].map(replace)
df['Final stake (%)'] = df['Final stake (%)'].map(replace)
df = df[df['Acquired stake (%)'] < 50]
df = df[df['Final stake (%)'] < 50]

shape_with_acquired_stake_over_50 = shape_without_unknown_acquired_stake - df.shape[0]
shape_acquired_stake_under_50 = df.shape[0]

#suppressoin des acquisitions sans date
df.dropna(axis=0,how='all', subset = ['Completed date','Assumed completion date'], inplace=True)
shape_acquisition_with_date =  shape_acquired_stake_under_50- df.shape[0]
shape_acquisition_with_date = df.shape[0]


#supprimer les acquisitions où le seller et le buyer sont les mêmes
df = df[df['Target BvD ID number'] != df['Acquiror BvD ID number']]
shape_acquiror_egal_target =  shape_acquisition_with_date - df.shape[0]
shape_without_acquiror_egal_target = df.shape[0]

# suppressions des acquisitions pour lesquelles il manque des données sur le pays des Target et des acquirors
df = df[(df['Target country code'].notna())|(df['Acquiror country code'].notna())]
# on ne garde que les acquisitions domestiques 
df = df[(df['Target country code'] == df['Acquiror country code'])]
shape_domestic = df.shape[0]
shape_cross_country = shape_without_acquiror_egal_target - df.shape[0]

#réindexer
df.reset_index(drop = True, inplace = True)

#création d'une nouvelle colonne Completed or assumed date contenant la date  
df['Completed or assumed date'] = df['Completed date']
for i in range(df.shape[0]):
    if type(df['Completed date'][i]) == float:
        df['Completed or assumed date'][i] = df['Assumed completion date'][i]


/var/folders/td/010hh4vx7gl3pcc1zz9bfdlh0000gn/T/ipykernel_38513/617938333.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Completed or assumed date'][i] = df['Assumed completion date'][i]


**Step 3:** \
From Zephir, we extracted SIC codes and Nace codes of the target companies, acquired companies and their parents. SIC Codes and Nace codes are needed in order to determine wether a deal is horizontal or not. \

We only collected the data of the companies of the top 30 countries with the most deals.\
Indeed, as extracting data from Orbis is very time consumming, and because regressions can only be done on countries with a lot of deals, we decided not to extract data about all the companies referenced on Orbis. 

In [5]:
# list of the 30 first countries.
List_major_countries = df['Target country code'].value_counts()[:30]
# list of the target companies
list_target = df[df['Target country code'].isin(List_major_countries.index)]['Target BvD ID number'].values
# list of the acquiror companies
list_acquiror = df[df['Acquiror country code'].isin(List_major_countries.index)]['Acquiror BvD ID number'].values

Dowloading the data, we obtained 4 excel databases. 2 regarding the target companies and 2 other regarding the acquiror companies and their parent companies. They are un the 'Zephir_data' folder. 

In [6]:
target1 = pd.read_excel('Zephir_data/Target NACE1.xls', 'Results')
target2 = pd.read_excel('Zephir_data/Target NACE2.xls', 'Results')

acquiror_parent2 = pd.read_excel('Zephir_data/Acquiror & CHS - NACE2.xls', 'Results')
acquiror_parent1 = pd.read_excel('Zephir_data/Acquiror & CSH NACE1.xls', 'Results')

WARNING *** file size (8068376) not 512 + multiple of sector size (512)
WARNING *** file size (10696885) not 512 + multiple of sector size (512)
WARNING *** file size (7190425) not 512 + multiple of sector size (512)
WARNING *** file size (9734975) not 512 + multiple of sector size (512)


The aim was now : \
1 - to concatenate target1 with target2, and acquiror_parent1 with acquiror_parent2. \
2 - to join target with data on Target BvD ID number, and acquiror_parent on Acquiror BvD ID number. 

In [7]:
# concaténation fichiers targets et acquirors
target = pd.concat([target1,target2], ignore_index=True)
acquiror_parents = pd.concat([acquiror_parent1,acquiror_parent2])

# Nettoyage de Target et Acquiror
target.dropna(axis=0,subset=['NACE Rev. 2, core code (4 digits)','NACE Rev. 2, secondary code(s)'], how='all', inplace=True)
target['Target BvD ID number'] = target['BvD ID number']
target = target[['Target BvD ID number','NACE Rev. 2, core code (4 digits)','NACE Rev. 2, secondary code(s)']]
target.set_index('Target BvD ID number',inplace=True)
acquiror_parents['Acquiror BvD ID number'] = acquiror_parents['BvD ID number']
acquiror_parents.dropna(axis=0, subset=['NACE Rev. 2, core code (4 digits)','NACE Rev. 2, secondary code(s)','CSH - NACE,\nCore code'], how='all', inplace=True)
acquiror_parents = acquiror_parents[['Acquiror BvD ID number','NACE Rev. 2, core code (4 digits)','NACE Rev. 2, secondary code(s)','CSH - NACE,\nCore code']]
acquiror_parents.set_index('Acquiror BvD ID number', inplace=True)
acquiror_parents.rename(columns={'NACE Rev. 2, core code (4 digits)':'primary acquiror NACE code',
 'NACE Rev. 2, secondary code(s)':'secondary acquiror NACE code(s)',
 'CSH - NACE,\nCore code':'parent NACE code(s)'}, inplace=True)

 # création d'un double index pour df
df.set_index(['Target BvD ID number','Acquiror BvD ID number'],inplace=True)

# join table df and target on Target BvD ID number
df = df.join(target, on='Target BvD ID number')
#join table df and acquiror_parents on Acquiror BvD ID number
df = df.join(acquiror_parents, on='Acquiror BvD ID number')


Now was the time to remove acquisitions made by investors. Are considered as acquisition made by investors : 
- acquisitions where the acquiror is an investor 
- acquisitions where the parent company of the acquiror is an investor 

In [8]:
# creation 1 colone Target NACE Code(s) et 1 colonne Acquiror NACE Code(s)
df['Acquiror NACE code(s)'] = df['primary acquiror NACE code'] 
df['Target NACE code(s)'] = df['NACE Rev. 2, core code (4 digits)'] 
for i in range(df.shape[0]):
    if type(df['Acquiror NACE code(s)'][i]) == str:
        if type(df['secondary acquiror NACE code(s)'][i]) == str:
            df['Acquiror NACE code(s)'][i] += '\n' + df['secondary acquiror NACE code(s)'][i]
    else :
        if type(df['secondary acquiror NACE code(s)'][i]) == str:
            df['Acquiror NACE code(s)'][i] = df['secondary acquiror NACE code(s)'][i]
    if type(df['Target NACE code(s)'][i]) == str:
        if type(df['NACE Rev. 2, secondary code(s)'][i]) == str:
            df['Target NACE code(s)'][i] += '\n' + df['NACE Rev. 2, secondary code(s)'][i]
    else :
        if type(df['NACE Rev. 2, secondary code(s)'][i]) == str:
            df['Target NACE code(s)'][i] = df['NACE Rev. 2, secondary code(s)'][i]


df2 = copy.deepcopy(df)
shape_after_join = df2.shape[0]
df2.drop(['NACE Rev. 2, core code (4 digits)','NACE Rev. 2, secondary code(s)','primary acquiror NACE code','secondary acquiror NACE code(s)'], axis=1, inplace=True)

#on retire les acquisiton lorsqu'on ne connaît pas le SIC code et le Nace code des Targets ou des Acquiror
df2.dropna(axis = 0, subset=['Target NACE code(s)','Target US SIC code(s)'], how = 'all', inplace = True)
df2.dropna(axis = 0, subset=["Acquiror NACE code(s)", 'Acquiror US SIC code(s)'], how = 'all', inplace = True)


# mise en forme des Nace dodes et des SIC codes
def fonction_NACE(x):
    if type(x) == str:
        try : 
            x = x.split('\n')
        except :
            x=np.nan
    else : 
        x = [x]
    return x

def fonction_SIC(x):
    if type(x) == str:
        x = x.split('/')
    else :
        x = [x]
    return x
df2['Acquiror NACE code(s)'] = df2['Acquiror NACE code(s)'].map(fonction_NACE)
df2['Target NACE code(s)'] = df2['Target NACE code(s)'].map(fonction_NACE)
df2['parent NACE code(s)'] = df2['parent NACE code(s)'].map(fonction_NACE)
df2['Target US SIC code(s)'] = df2['Target US SIC code(s)'].map(fonction_SIC)
df2['Acquiror US SIC code(s)'] = df2['Acquiror US SIC code(s)'].map(fonction_SIC)


# retirer les acquisitions faites par des investisseurs
# les acquisitions faites par des investeurs sont celles ou l'acquiror ET le parent de l'acquiror sont des investisseurs
def find_investor_ac(x):
    for element in x:
        if type(element) == str :
            if '64' in element[:2]:
                x = False
            else : 
                x = True
        else : 
            x = False
    return x

def find_investor_par(x):
    if type (x) == str:
        if '64' in x[:2]:
            x = False
        else :
            x = True
    else : 
        x = False
    return x

mask_investor_1 = df2['Acquiror NACE code(s)'].map(find_investor_ac)
mask_investor_2 = df2['parent NACE code(s)'].map(find_investor_par)
liste = [] 
for i, element in enumerate(mask_investor_1):
    if element == False :
        if element == mask_investor_2[i]:
            liste.append(False)
        else :
            liste.append(True)
    else : 
        liste.append(True)
df2 = df2[liste]
shape_acquisition_by_investor = shape_after_join - df2.shape[0]
shape_without_investor = df2.shape[0]

/var/folders/td/010hh4vx7gl3pcc1zz9bfdlh0000gn/T/ipykernel_38513/2037445781.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Acquiror NACE code(s)'][i] += '\n' + df['secondary acquiror NACE code(s)'][i]
/var/folders/td/010hh4vx7gl3pcc1zz9bfdlh0000gn/T/ipykernel_38513/2037445781.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Target NACE code(s)'][i] += '\n' + df['NACE Rev. 2, secondary code(s)'][i]
/var/folders/td/010hh4vx7gl3pcc1zz9bfdlh0000gn/T/ipykernel_38513/2037445781.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

Then, we decided to create 4 different tables : 
- horizontal deals with data about the target company  : Target_horizontal 
- horizontal deals with data about the acquiror company : Acquiror_horizontal 
- non horizontal deals with data about the target company : Target_non_horizontal 
- non horizontal deals with data about the acquiror company : Acquiror_non_horizontal 

In order to do so, it is needed to separate horizontal from non_horizontal acquisitions.\
A deal is considered horizontal when : 
- the target and the acquiror evolve in the same sector of activity (the target SIC code or Nace code  and acquiror SIC code or Nace code are the same). 
- the target and the parent of the acquiror evolve in the same sector of activity (target Nace code and acquiror parent Nace code are the same).


In [9]:
# horizontal acquisitions
# sont considérées comme horizontales les acquisitions où :
# - target et acquiror évoluent dans le même secteur d'activité (Nace Code + SIC Code)
# - target et parent de l'acquiror évoluent dans le même secteur d'activité (Nace code)
df2['horizontal'] = True 
def is_horizontal(x):
    parent = -4
    target = -2
    acquiror = -3
    horizontal = -1
    target_sic = 19
    acquiror_sic = 27
    for element in x[acquiror]:
        if element in x[target] :
            x[horizontal] = True 
        else : 
            x[horizontal] = False
    if type(x[parent]) == str:
        if x[parent] in x[target]:
            x[horizontal] = True

    for element in x[acquiror_sic]:
        if element in x[target_sic]:
            x[horizontal] = True 
    return x

mask_horizontal = df2.apply(is_horizontal, axis = 1) # True si horizontal et False sinon
mask_horizontal = mask_horizontal['horizontal']
df_horizontal = df2[mask_horizontal]
shape_is_horizontal = df_horizontal.shape[0]

df_horizontal.reset_index(inplace = True)




# Non horizontal acquisitions
mask_non_horizontal = []
for i,element in enumerate(mask_horizontal):
    mask_non_horizontal.append(not element)
df_non_horizontal = df2[mask_non_horizontal]
shape_non_horizontal = df_non_horizontal.shape[0]

df_non_horizontal.reset_index(inplace = True)
df_non_horizontal


,Target BvD ID number,Acquiror BvD ID number,Unnamed: 0,Unnamed: 0.1,Deal Number,Deal type,Deal status,Deal value\nm EUR,Completed date,Assumed completion date,...,Acquiror ISH Name,Acquiror ISH BvD number,Acquiror ISH ISO country code,Acquiror ISH Direct %,Acquiror ISH Total %,Completed or assumed date,parent NACE code(s),Acquiror NACE code(s),Target NACE code(s),horizontal
0,FI09238664,FI09507502,11,12.0,5.397950e+05,Minority stake 40%,Completed,NaN,2007-05-22,NaN,...,NaN,NaN,NaN,NaN,NaN,2007-05-22,[nan],[5813],[5320],True
1,SE5560499690,SE5561980128,12,13.0,5.398480e+05,Minority stake increased to 5.5%,Completed,NaN,2007-05-23,NaN,...,SWEDBANK ROBUR AB,SE5561103895,SE,100.00,100.00,2007-05-23,"[6419, 6619]",[6630],[5829],True
2,RU99263319,RU09801500,13,14.0,5.398580e+05,Minority stake 25%,Completed,NaN,2007-05-18,NaN,...,NaN,NaN,NaN,NaN,NaN,2007-05-18,[nan],"[6419, 6492, 6499, 6612, 6619]","[6491, 6499, 6810, 6820, 6830, 6900, 7310, 771...",True
3,RU07506317,RU56467052,14,15.0,5.398640e+05,Minority stake 20.16%,Completed,NaN,2007-05-21,NaN,...,ROSTEC,RU94137372,RU,100.00,100.00,2007-05-21,"[8411, 8411, -]","[4669, 4612, 4614, 4669, 4675, 4690, 6311]","[3030, 3030]",True
4,RU12169000,RU57497960,16,17.0,5.399130e+05,Minority stake 6.87%,Completed,NaN,2007-05-18,NaN,...,NaN,NaN,NaN,NaN,NaN,2007-05-18,[nan],"[4720, 1071, 1072, 4611, 4612, 4621]","[6820, 2211, 4520, 4530, 4930, 4941, 7711, 7739]",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47657,US208481962,US470813844,9986,9987.0,1.601447e+09,Minority stake increased from 2.531% to 6.445%,Completed,108.09*,2012-12-31,NaN,...,NaN,NaN,NaN,NaN,NaN,2012-12-31,[nan],[6500],"[2910, 2813, 2899, 2932]",True
47658,CN31136PC,CN9381915283,9987,9988.0,1.943250e+09,Minority stake 4.99%,Completed,108.05*,2021-02-08,NaN,...,JIANGXI TRANSPORT AND SHIPPING OFFICE,CN9457380403,CN,100.00,100.00,2021-02-08,[nan],[4221],"[4211, 5229]",True
47659,TH0105552133918,TH0115557021034,9989,9990.0,1.943236e+09,Minority stake increased from 4.71% to 25.23%,Completed Assumed,108.04*,NaN,2021-01-29,...,NaN,NaN,NaN,NaN,NaN,2021-01-29,[nan],[6832],"[4110, 6820]",True
47660,ZA200801569606,ZA195700197906,9993,9994.0,1.941624e+09,Minority stake increased from 24.52% to 29.38%,Completed,108.01*,2021-11-29,NaN,...,NaN,NaN,NaN,NaN,NaN,2021-11-29,[nan],"[0729, 2441]",[2892],True


For each new database, we chose to keep only some columns. 
For Target_horizontal and Target_non_horizontal, we kept : "Deal value\nm EUR", "Target country code", "Target BvD ID number", "Acquiror country code", "Completed date", "Acquired stake (%)", "Final stake (%)" \
For Acquiror_horizontal and Acquiror_non_horizontal, we kept : "Deal value\nm EUR", "Target country code", "Acquiror country code", "Acquiror BvD ID number", "Completed date", "Acquired stake (%)", "Final stake (%)"

In [10]:
Target_horizontal = df_horizontal[["Deal value\nm EUR", "Target country code", "Target BvD ID number", "Acquiror country code", "Completed date", "Acquired stake (%)", "Final stake (%)"]]
Acquiror_horizontal = df_horizontal[["Deal value\nm EUR", "Target country code", "Acquiror country code", "Acquiror BvD ID number", "Completed date", "Acquired stake (%)", "Final stake (%)"]]
Target_non_horizontal = df_non_horizontal[["Deal value\nm EUR", "Target country code", "Target BvD ID number", "Acquiror country code", "Completed date", "Acquired stake (%)", "Final stake (%)"]]
Acquiror_non_horizontal = df_non_horizontal[["Deal value\nm EUR", "Target country code", "Acquiror country code", "Acquiror BvD ID number", "Completed date", "Acquired stake (%)", "Final stake (%)"]]

In [11]:
# exportation to .csv
Target_horizontal.to_csv('target_horizontal.csv', index = True)
Acquiror_horizontal.to_csv('acquiror_horizontal.csv', index = True)
Target_non_horizontal.to_csv('target_non_horizontal.csv', index = True)
Acquiror_non_horizontal.to_csv('acquiror_non_horizontal.csv', index = True)

**Collecting and extracting data from Orbis** \
From now one, we had to find data about the companies : number of employees, R&D expenses ... Those data were available on Orbis through a remote Desktop. We download data regarding : ......... for every company of the 30 first countries. Those databases are in the folder 'DATABASE_Orbis', sorted country by country. 

In [12]:
#On charge les 4 fichiers
df_H_target = pd.read_csv("target_horizontal.csv")
df_NH_target = pd.read_csv("target_non_horizontal.csv")
df_H_acquiror = pd.read_csv("acquiror_horizontal.csv")
df_NH_acquiror = pd.read_csv("acquiror_non_horizontal.csv")

The **first step** was to concatenate the excel files in 'DATABASE_Obis'.

In [13]:
path = "DATABASE_Orbis"
folder = os.listdir(path)
data_r_d = pd.DataFrame()
for file in folder :
    data = pd.read_csv(f'{path}/{file}')
    data_r_d = data_r_d.append(data)
data_r_d
# exportation to .csv
data_r_d.to_csv('data_r_d.csv')
data_r_d = pd.read_csv('data_r_d.csv')

**Second Step**: \
we had to only keep the date year of the orbis database ('data_r_d')

In [14]:
# fonction that only keep the date year. 
def keep_year_orbis(x):
    x[2] = str(x[2])[:4]
    return x

def keep_year_zephir(x):
    x[5] = str(x[5])[:4]
    return x



#On ne garde que l'année et on retire le mois et le jour
data_r_d = data_r_d.apply(keep_year_orbis, axis = 1)
data_r_d.set_index(["bvdid","closing_date"], inplace=True) 
data_r_d.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
data_r_d1 = data_r_d.reset_index()

**third step** \
join the orbis database with the zephir databases.  
- 1) join inner with the series of bvdid of orbis and zephir tables on bvdid to keep only the companies that are in both.
- 2) join inner with the orbis database the series of bvdid of the table created just before on bvdid to keep only the companies that are in both.
- 3) join on bvdid and closing date of the table created in 1) with the Orbis table in order to have every deal with all their r&d informations for the year of the deal. 
- 4)join on bvdid and closing date of the table created in 3) with the Orbis table in order to have additionnal pieces of information about the company for the year without deal. 

In [15]:
def creation_final_table(data, name, type) : 

    
    data.rename(columns = {f'{type} BvD ID number' : "bvdid", 'Completed date' : "closing_date", 'Unnamed: 0': 'Deal_number'}, inplace = True)
    data.dropna(axis = 0,subset=["closing_date"], how = 'any',inplace = True)
    #On ne garde que l'année et on retire le mois et le jour
    data = data.apply(keep_year_zephir, axis = 1)

    data.set_index(["bvdid","closing_date"], inplace=True)
    data1 = data.reset_index()

    # join inner sur la serie des bvdid data_r_d et data pour ne garder que les compagnies présentent dans les deux tables. 
    serie_bvdid_r_d = data_r_d1['bvdid']
    # join
    data_deals = data1.merge(serie_bvdid_r_d, on =["bvdid"], how = 'inner')
    data_deals.reset_index(inplace=True)
    data_deals.drop_duplicates(subset = ['bvdid','closing_date',"Deal_number",'Deal value\nm EUR',f'{type} country code','Acquiror country code','Acquired stake (%)','Final stake (%)'], inplace = True)
    #création d'un deal_number
    data_deals.reset_index(inplace = True)
    data_deals.rename(columns = {'level_0':'deal_number'}, inplace = True)
    data_deals_bvdid = data_deals['bvdid']

    # join inner sur data_r_d et avec bvdid présent dans data et data_r_d pour ne garder que les compagnies présentent dans les deux tables. 
    data_r_d_deals = data_r_d1.merge(data_deals_bvdid, on = ['bvdid'], how = 'inner')
    data_r_d_deals.drop_duplicates(subset = list(data_r_d_deals.columns), inplace = True)

    # join sur les bvdid et closing date de la table précédente avec data_r_d pour avoir tous les deals avec leur informations r&d de l'année correspondante. 
    data_deals.set_index(["bvdid","closing_date"], inplace=True)
    # join
    data_deals_left = data_deals.join(data_r_d, on =["bvdid",'closing_date'], how = 'left')
    data_deals_left.reset_index(inplace=True)

    # data sur data_r_d_deals
    data_r_d_deals.set_index(["bvdid","closing_date"], inplace=True)
    # join 
    data_deals_right = data_r_d_deals.join(data, on = ["bvdid","closing_date"], how = 'left')
    data_deals_right.reset_index(inplace = True)

    # reindéxation des colonnes pour que ce soit les même hez right et left
    data_deals_left.drop(axis = 1, labels=['index'], inplace = True)
    data_deals_left = data_deals_left.reindex(columns = list(data_deals_right.columns))
    # concaténation de right et left
    data_deals_final = pd.concat([data_deals_left,data_deals_right])
    #drop duplicates
    data_deals_final.drop_duplicates(inplace = True)
    # exportation to .csv
    data_deals_final.to_csv(f'{name}.csv')

In [16]:
creation_final_table(df_H_target, name = 'horizontal_domestic_target', type = 'Target')
creation_final_table(df_NH_target, name = 'non_horizontal_domestic_target', type = 'Target')
creation_final_table(df_H_acquiror, name = 'horizontal_domestic_acquiror', type = 'Acquiror')
creation_final_table(df_NH_acquiror, name = 'non_horizontal_domestic_acquiror', type = 'Acquiror')

In [21]:
# importation
df_H_target = pd.read_csv('horizontal_domestic_target.csv')
df_NH_target = pd.read_csv('non_horizontal_domestic_target.csv')
df_H_acquiror = pd.read_csv('horizontal_domestic_acquiror.csv')
df_NH_acquiror = pd.read_csv('non_horizontal_domestic_acquiror.csv')

In [22]:
def deal_value(x):
    if type (x) == str:
        if '*' in x:
            x = x[:-1]
        x = float(x)
    return x

df_H_target['Deal value\nm EUR'] = df_H_target['Deal value\nm EUR'].map(deal_value)

In [32]:
# normalization 
from sklearn import preprocessing
serie = df_H_target['r_d_expenses_operating_revenue']
df_H_target['r_d_expenses_operating_revenue']=(serie-serie.mean())/serie.std()
def normalization(data):
    serie = data['r_d_expenses_operating_revenue']
    data['r_d_expenses_operating_revenue']=(serie-serie.mean())/serie.std()
#    data['r_d_expenses_operating_revenue'] = preprocessing.normalize(np.array(data['r_d_expenses_operating_revenue']))[0,:]
#    data['research_development_expenses'] = preprocessing.normalize(np.array(data['research_development_expenses']))[0,:]
    #data['Deal value\nm EUR'] = preprocessing.normalize(np.array(data['Deal value\nm EUR']))[0,:]
    #data['number_of_employees'] = preprocessing.normalize(np.array(data['number_of_employees']))[0,:]
    #data['ebitda'] = preprocessing.normalize(np.array(data['ebitda']))[0,:]
    #data['Score-protecting minority investors'] = preprocessing.normalize(np.array(data['Score-protecting minority investors']))[0,:]

normalization(df_H_target)
normalization(df_NH_target)
normalization(df_H_acquiror)
normalization(df_NH_acquiror)

**Aggregation**

In [33]:
def aggregate(data):
    # keep only corporate firms
    data = data[data['type_of_entity'] == 'Corporate']


    # dummy variables for acquired stake
    data['Acquired stake (%)_0_10'] = np.where(data['Acquired stake (%)'] <= 10, 1,0)
    data['Acquired stake (%)_10_20'] = np.where((data['Acquired stake (%)'] <= 20) & (data['Acquired stake (%)'] > 10), 1,0)
    data['Acquired stake (%)_20_30'] = np.where((data['Acquired stake (%)'] <= 30) & (data['Acquired stake (%)'] > 20), 1,0)
    data['Acquired stake (%)_30_40'] = np.where((data['Acquired stake (%)'] <= 40) & (data['Acquired stake (%)'] > 30), 1,0)
    data['Acquired stake (%)_40_50'] = np.where((data['Acquired stake (%)'] <= 50) & (data['Acquired stake (%)'] > 40), 1,0)
    data['Acquired stake (%)_0_25'] = np.where(data['Acquired stake (%)'] <= 25 , 1,0)
    data['Acquired stake (%)_25_50'] = np.where(data['Acquired stake (%)'] > 25, 1,0)
    data['Acquired stake (%)_0_50'] = np.where(data['Acquired stake (%)'] > 0, 1,0)

    def fonction_acquired (x, i):
        Acquired_stake = x[13]
        deal_number = x[9]
        if deal_number in s : 
            x[-i] = 0
        else : 
            if x[-i] == 1:
                s.add(deal_number)
        return x

    for k in range(1,8):
        s = set()
        data = data.apply(lambda x: fonction_acquired(x,k), axis=1)

       # sum of deal value for each slice
    data['Total deal value for Acquired stake (%)0_10'] = np.where(data['Acquired stake (%)'] <= 10, data['Deal value\nm EUR'],1,0)
    data['Total deal value for Acquired stake (%)_10_20'] = np.where((data['Acquired stake (%)'] <= 20) & (data['Acquired stake (%)'] > 10), data['Deal value\nm EUR'],1,0)
    data['Total deal value for Acquired stake (%)_20_30'] = np.where((data['Acquired stake (%)'] <= 30) & (data['Acquired stake (%)'] > 20), data['Deal value\nm EUR'],1,0)
    data['Total deal value for Acquired stake (%)_30_40'] = np.where((data['Acquired stake (%)'] <= 40) & (data['Acquired stake (%)'] > 30), data['Deal value\nm EUR']1,,0)
    data['Total deal value for Acquired stake (%)_40_50'] = np.where((data['Acquired stake (%)'] <= 50) & (data['Acquired stake (%)'] > 40), data['Deal value\nm EUR']1,,0)
    data['Total deal value for Acquired stake (%)_0_25'] = np.where(data['Acquired stake (%)'] <= 25 , data['Deal value\nm EUR'],1,0)
    data['Total deal value for Acquired stake (%)_25_50'] = np.where(data['Acquired stake (%)'] > 25, data['Deal value\nm EUR'],1,0)
    data['Total deal value for Acquired stake (%)_0_50'] = np.where(data['Acquired stake (%)'] > 0, data['Deal value\nm EUR'],1,0)

    def fonction_total_deal(x, i):
        Acquired_stake = x[13]
        deal_number = x[9]
        if deal_number in s : 
            x[-i] = 0
        else : 
            if x[-i] != 0:
                s.add(deal_number)
        return x

    for k in range(1,8):
        s = set()
        data = data.apply(lambda x: fonction_total_deal(x,k), axis=1)

    
    # dummy variables for final stake
    data['Final stake (%)_0_10'] = np.where(data['Final stake (%)'] <= 10, 1,0)
    data['Final stake (%)_10_20'] = np.where((data['Final stake (%)'] <= 20) & (data['Final stake (%)'] > 10), 1,0)
    data['Final stake (%)_20_30'] = np.where((data['Final stake (%)'] <= 30) & (data['Final stake (%)'] > 20), 1,0)
    data['Final stake (%)_30_40'] = np.where((data['Final stake (%)'] <= 40) & (data['Final stake (%)'] > 30), 1,0)
    data['Final stake (%)_40_50'] = np.where((data['Final stake (%)'] <= 50) & (data['Final stake (%)'] > 40), 1,0)
    data['Final stake (%)_0_25'] = np.where(data['Final stake (%)'] <= 25 , 1,0)
    data['Final stake (%)_25_50'] = np.where(data['Final stake (%)'] > 25, 1,0)

    

    def fonction_final (x, i):
        final_stake = x[14]
        deal_number = x[9]
        if deal_number in s : 
            x[-i] = 0
        else : 
            if x[-i] == 1:
                s.add(deal_number)
        return x

    for k in range(1,8):
        s = set()
        data = data.apply(lambda x: fonction_final(x,k), axis=1)


    # mettre un country code à toutes les lignes en se basant sur celui du bvdid
    data['countrycode'] = data['bvdid']
    def bvdid_to_country_code(x):
        x = x[:2]
        return x
    data['countrycode'] = data['countrycode'].map(bvdid_to_country_code)

    # aggregate each row where country and date are the same
    data.drop(['Deal_number','Acquired stake (%)','Final stake (%)','Unnamed: 0'], axis = 1, inplace=True)
    df_by_country_date = data.groupby(['countrycode','closing_date']).sum()
    return df_by_country_date

#Apply the function to the tables
new_df_NH_target = aggregate(df_NH_target)
new_df_H_target = aggregate(df_H_target)
new_df_H_acquiror = aggregate(df_H_acquiror)
new_df_NH_acquiror = aggregate(df_NH_acquiror)

/var/folders/td/010hh4vx7gl3pcc1zz9bfdlh0000gn/T/ipykernel_38513/1142177735.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Acquired stake (%)_0_10'] = np.where(data['Acquired stake (%)'] <= 10, 1,0)
/var/folders/td/010hh4vx7gl3pcc1zz9bfdlh0000gn/T/ipykernel_38513/1142177735.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Acquired stake (%)_10_20'] = np.where((data['Acquired stake (%)'] <= 20) & (data['Acquired stake (%)'] > 10), 1,0)
/var/folders/td/010hh4vx7gl3pcc1zz9bfdlh0000gn/T/ipyk

**ADD REFORM INFORMATION**

In [34]:
#Add the 2 columns with reforms

#Add country protecting index :

df_reforms = pd.read_csv("score_protect_ms_final.csv")
df_reforms.drop(['Unnamed: 0','Unnamed: 0.1', 'Economy'], axis = 1, inplace = True)
df_reforms.rename(columns = {"Country code" : "countrycode", "DB year" : "closing_date"}, inplace = True)
df_reforms.set_index(["countrycode","closing_date"], inplace = True)

#Add country reform :
df_ref = pd.read_csv("country_reform.csv", sep = ";")
df_ref.drop(['Coutry '], axis = 1, inplace = True)
df_ref["is_reform"] = True
df_ref.rename(columns = {'Year ': "closing_date", "Country code" : "countrycode"}, inplace = True)
df_ref.set_index(["closing_date", "countrycode"], inplace = True)
df_reforms = df_reforms.join(df_ref, on = ["closing_date", "countrycode"])

#Add the columns of cumulative reform
time = 0
for i in range(1,df.shape[0]-1):
    country = df['countrycode'][i]
    country_before = df['countrycode'][i-1]
    if country == country_before :
        if df['is_reform'][i] == 1:
            time +=1
        if time == 1 :
            df['reform1'][i] = 1
        if time == 2 :
            df['reform2'][i] = 1
            df['reform1'][i] = 1

    else :
        time = 0
        if df['is_reform'][i] == 1:
            time +=1
        if time == 1 :
            df['reform1'][i] = 1
        if time == 2 :
            df['reform2'][i] = 1
            df['reform1'][i] = 1


#Join final table with the one containing reform score
new_df_H_target = new_df_H_target.join(df_reforms, on = ["countrycode","closing_date"], how = "left")
new_df_NH_target = new_df_NH_target.join(df_reforms, on = ["countrycode","closing_date"], how = "left")
new_df_H_acquiror = new_df_H_acquiror.join(df_reforms, on = ["countrycode","closing_date"], how = "left")
new_df_NH_acquiror = new_df_NH_acquiror.join(df_reforms, on = ["countrycode","closing_date"], how = "left")

#Creation of final csv
new_df_H_target.to_csv("H_target_final.csv")
new_df_H_acquiror.to_csv("H_acquiror_final.csv")
new_df_NH_target.to_csv("NH_target_final.csv")
new_df_NH_acquiror.to_csv("NH_acquiror_final.csv")